<a href="https://colab.research.google.com/github/joaopaiva/trading-strategies/blob/main/BacktraderMainReversionStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# suppress output
%%capture
!pip install yfinance pandas_ta

In [43]:
import numpy as np
import yfinance as yf
from datetime import datetime, date, time, timedelta
import pandas as pd
import pandas_ta as ta
# import pandas_datareader.data as web
import matplotlib.pyplot as plt
from datetime import date

plt.style.use('fivethirtyeight')
yf.pdr_override()

# Define the ticker symbol and start and end dates
stocksymbols = ['BNB-USD']
# start_date = '2022-01-01'
# end_date = '2022-02-18'
start_date = datetime.today() - timedelta(days=6)
start_date = start_date.strftime("%Y-%m-%d")
end_date = datetime.today()


In [45]:
def getHistoricalData(stocks = stocksymbols ,start = start_date , end = end_date):
    # Get the data for the ticker
    ticker_data = yf.Ticker(stocksymbols[0])

    # Get the historical prices for the ticker
    data = ticker_data.history(start=start_date, end=end_date, interval='15m')

    # data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

historical_data = getHistoricalData(stocksymbols)

# Calculate the rolling mean and standard deviation
rolling_mean = historical_data['Close'].rolling(window=30).mean()
rolling_std = historical_data['Close'].rolling(window=30).std()

# Calculate the upper and lower Bollinger Bands
upper_band = rolling_mean + (2 * rolling_std)
lower_band = rolling_mean - (2 * rolling_std)

# Identify when the price crosses the bands
historical_data['buy_signal'] = (historical_data['Close'] < lower_band)
historical_data['sell_signal'] = (historical_data['Close'] > upper_band)

# Take positions based on the signals
positions = pd.Series(0, index=historical_data.index)
positions[historical_data['buy_signal']] = 1
positions[historical_data['sell_signal']] = -1

# Calculate returns
historical_data['returns'] = positions.shift(1) * (historical_data['Close'] - historical_data['Close'].shift(1))

# Calculate cumulative returns
historical_data['cumulative_returns'] = historical_data['returns'].cumsum()

# print(historical_data)

In [55]:
historical_data['SMA 30'] = ta.sma(historical_data['Close'],30)
historical_data['SMA 100'] = ta.sma(historical_data['Close'],100)
#SMA BUY SELL
#Function for buy and sell signal
def buy_sell(historical_data):
    signalBuy = []
    signalSell = []
    position = False 

    for i in range(len(historical_data)):
        # print(historical_data)
        if historical_data['SMA 30'][i] > historical_data['SMA 100'][i]:
            if position == False :
                signalBuy.append(historical_data['Close'][i])
                signalSell.append(False)
                position = True
            else:
                signalBuy.append(False)
                signalSell.append(False)
        elif historical_data['SMA 30'][i] < historical_data['SMA 100'][i]:
            if position == True:
                signalBuy.append(False)
                signalSell.append(historical_data['Close'][i])
                position = False
            else:
                signalBuy.append(False)
                signalSell.append(False)
        else:
            signalBuy.append(False)
            signalSell.append(False)
    return pd.Series([signalBuy, signalSell])

In [ ]:
historical_data

In [56]:
# historical_data
historical_data.reset_index(inplace = True,drop = True)
buy_sell(historical_data)
historical_data['Buy_Signal_price'], historical_data['Sell_Signal_price'] = buy_sell(historical_data)
historical_data

           Open        High         Low       Close   Volume  Dividends  \
0    310.605621  310.717316  310.186646  310.310669    64416        0.0   
1    310.313782  310.313782  310.169800  310.224762    23168        0.0   
2    310.214294  310.214294  309.390167  309.399139        0        0.0   
3    309.422302  309.452972  309.333984  309.421844   137344        0.0   
4    309.429382  309.585846  309.428680  309.585846        0        0.0   
..          ...         ...         ...         ...      ...        ...   
622  313.953339  314.637299  313.953339  314.619965  2258560        0.0   
623  314.602966  314.635864  314.594513  314.606476   280640        0.0   
624  314.661407  315.549561  314.568115  315.549561  1884544        0.0   
625  315.666229  315.931824  315.635376  315.911682  1405632        0.0   
626  316.149902  316.149902  315.891968  315.891968   432256        0.0   

     Stock Splits  buy_signal  sell_signal   returns  cumulative_returns  \
0             0.0      

KeyError: ignored

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
ax.plot(historical_data['Adj Close'] , label = stocksymbols[0] ,linewidth=0.5, color='blue', alpha = 0.9)
ax.plot(historical_data['SMA 30'], label = 'SMA30', alpha = 0.85)
ax.plot(historical_data['SMA 100'], label = 'SMA100' , alpha = 0.85)
ax.scatter(historical_data.index , historical_data['Buy_Signal_price'] , label = 'Buy' , marker = '^', color = 'green',alpha =1 )
ax.scatter(historical_data.index , historical_data['Sell_Signal_price'] , label = 'Sell' , marker = 'v', color = 'red',alpha =1 )
ax.set_title(stocksymbols[0] + " Price History with buy and sell signals",fontsize=10, backgroundcolor='blue', color='white')
ax.set_xlabel(f'{start_date} - {end_date}' ,fontsize=18)
ax.set_ylabel('Close Price INR (₨)' , fontsize=18)
legend = ax.legend()
ax.grid()
plt.tight_layout()
plt.show()